# Predicting house prices using k-nearest neighbors regression
In this notebook, you will implement k-nearest neighbors regression. You will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

# Fire up GraphLab Create

In [1]:
import graphlab

# Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [2]:
sales = graphlab.SFrame('kc_house_data_small.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\danqi\AppData\Local\Temp\graphlab_server_1560451314.log.0


This non-commercial license of GraphLab Create for academic use is assigned to dxiao@bloomu.edu and will expire on April 02, 2020.


# Import useful functions from previous notebooks

To efficiently compute pairwise distances among data points, we will convert the SFrame into a 2D Numpy array. First import the numpy library and then copy and paste `get_numpy_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    
    features_sframe=data_sframe[features]
    
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray=data_sframe[output]
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

We will also need the `normalize_features()` function from Week 5 that normalizes all feature columns to unit norm. Paste this function below.

In [5]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return feature_matrix/norms,norms

# Split data into training, test, and validation sets

In [6]:
(train_and_validation, test) = sales.random_split(.8, seed=1) # initial train/test split
(train, validation) = train_and_validation.random_split(.8, seed=1) # split training set into training and validation sets

# Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transform the training, test, and validation SFrames into Numpy arrays:

In [7]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [8]:
features_train, norms = normalize_features(features_train) # normalize training set features (columns)
features_test = features_test / norms # normalize test set by training set norms
features_valid = features_valid / norms # normalize validation set by training set norms

# Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1.

In [14]:
print features_test[0]

[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059    0.
  0.05102365  0.0116321   0.01564352  0.01362084  0.02481682  0.01350306
  0.          0.01345386 -0.01346927  0.01375926  0.0016225 ]


Now print the 10th row (index 9) of the training feature matrix. Again, you get an 18-dimensional vector with components between 0 and 1.

In [12]:
print features_test[9]

[ 0.01345102  0.01163464  0.02108718  0.00948186  0.00035378  0.02558851
  0.          0.          0.0116321   0.01390535  0.01093795  0.
  0.01371458  0.          0.01348525 -0.01346613  0.01041589  0.00128421]


***QUIZ QUESTION ***

What is the Euclidean distance between the query house and the 10th house of the training set? 

Note: Do not use the `np.linalg.norm` function; use `np.sqrt`, `np.sum`, and the power operator (`**`) instead. The latter approach is more easily adapted to computing multiple distances at once.

In [18]:
dis=(features_test[0]-features_test[9])**2
dis2=np.sum(dis)    
edis=np.sqrt(dis2)
print edis

0.0583528536451


# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

In [20]:
targeth=features_train[0:10]
queryh=features_test[0]
edisl=[]
for i in targeth:
    edis=np.sqrt(np.sum(targeth-i)**2)
    edisl.append(edis)
print edisl

[0.3573445431939819, 0.75761168409336699, 0.35452053041496689, 0.14699438426182526, 0.16789195293973599, 0.29299528799120977, 0.099361409683705895, 0.45764249890021164, 0.091153721443580527, 0.18128856447538375]


*** QUIZ QUESTION ***

Among the first 10 training houses, which house is the closest to the query house?

In [ ]:
#9th house closest.

It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [21]:
for i in xrange(3):
    print features_train[i]-features_test[0]
    # should print 3 vectors of length 18

[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
   2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
  -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
  -5.14364795e-03   6.69281453e-04]
[  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
   7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
  -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
  -2.89330197e-03   1.47606982e-03]
[  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
   1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
  -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
   3.72914476e-03   1.64764925e-03]


The subtraction operator (`-`) in Numpy is vectorized as follows:

In [22]:
print features_train[0:3] - features_test[0]

[[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
    2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
   -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
   -5.14364795e-03   6.69281453e-04]
 [  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
    7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
   -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
   -2.89330197e-03   1.47606982e-03]
 [  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
    1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
   -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
    3.72914476e-03   1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [23]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print results[0] - (features_train[0]-features_test[0])
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print results[1] - (features_train[1]-features_test[0])
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print results[2] - (features_train[2]-features_test[0])
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


Aside: it is a good idea to write tests like this cell whenever you are vectorizing a complicated operation.

# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house.

In [24]:
print len(features_train)


5527


In [25]:
diff=features_train[0:5527] - features_test[0]

To test the code above, run the following cell, which should output a value -0.0934339605842:

In [26]:
print diff[-1].sum() # sum of the feature differences between the query and last training house
# should print -0.0934339605842

-0.0934339605842


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [27]:
print np.sum(diff**2, axis=1)[15] # take sum of squares across each row, and print the 16th sum
print np.sum(diff[15]**2) # print the sum of squares for the 16th row -- should be same as above

0.00330705902879
0.00330705902879


With this result in mind, write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Assign the result to a variable `distances`.

**Hint**: Do not forget to take the square root of the sum of squares.

In [28]:
distances=np.sqrt(np.sum(diff**2, axis=1))

To test the code above, run the following cell, which should output a value 0.0237082324496:

In [29]:
print distances[100] # Euclidean distance between the query house and the 101th training house
# should print 0.0237082324496

0.0237082324496


Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [31]:
def getdistance(feature_train,target_house):
    trainl=len(feature_train)
    diff=feature_train[0:trainl] - target_house
    distances=np.sqrt(np.sum(diff**2, axis=1))
    return distances
    

In [37]:
target_house=features_test[2]
mydis=getdistance(features_train,target_house)
item=graphlab.SArray(xrange(len(mydis)))
rtable=graphlab.SFrame({'index':item,'distance':mydis})
print rtable

+------------------+-------+
|     distance     | index |
+------------------+-------+
| 0.0195447561632  |   0   |
|  0.068610347115  |   1   |
| 0.0216507855352  |   2   |
| 0.0128338471662  |   3   |
| 0.0199091754538  |   4   |
| 0.0191286493456  |   5   |
| 0.00543398935931 |   6   |
| 0.0293867516827  |   7   |
|  0.012479372926  |   8   |
| 0.0218369023013  |   9   |
+------------------+-------+
[5527 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [39]:
rtable.sort('distance')

distance,index
0.00286049526751,382
0.00322584038804,1149
0.00350215650158,4087
0.00359315377903,3142
0.00391857515673,2751
0.00401931061474,4556
0.00415816948089,3372
0.004185659353,4584
0.00426042031815,1666
0.00432702581392,2283


*** QUIZ QUESTIONS ***

1.  Take the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [42]:
pvalue=train[382]['price']
print pvalue

249000


In [188]:
pvalue2=train[382]['price']+train[1149]['price']+train[4087]['price']+train[3142]['price']
print pvalue2/4

413987


# Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [236]:
def k_nearest_neighbors(k, feature_train, features_query):
    
    undistance2knn=getdistance(feature_train,features_query)
    
    index=graphlab.SArray(xrange(len(feature_train)))
    rtable=graphlab.SFrame({'index':index,'distance':undistance2knn})
    rtable2=rtable.sort('distance')
    #print rtable2
    n_index=[]
    for i in range(k):
        n_index.append(rtable2[i]['index'])
    return n_index
        
    
    

*** QUIZ QUESTION ***

Take the query house to be third house of the test set (`features_test[2]`).  What are the indices of the 4 training houses closest to the query house?

In [207]:
target=features_test[2]
nhouses=k_nearest_neighbors(4,features_train,target)
print nhouses


+------------------+-------+
|     distance     | index |
+------------------+-------+
| 0.00286049526751 |  382  |
| 0.00322584038804 |  1149 |
| 0.00350215650158 |  4087 |
| 0.00359315377903 |  3142 |
| 0.00391857515673 |  2751 |
| 0.00401931061474 |  4556 |
| 0.00415816948089 |  3372 |
|  0.004185659353  |  4584 |
| 0.00426042031815 |  1666 |
| 0.00432702581392 |  2283 |
+------------------+-------+
[5527 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
[382L, 1149L, 4087L, 3142L]


## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**Hint**: You can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.

In [222]:
def predict_output_of_query(k, feature_train, output_train, features_query):
    kindex=k_nearest_neighbors(k, feature_train, features_query)
    print kindex
    output_sum=0
    for i in kindex:
        output_sum=output_sum+output_train[i]
        
    return output_sum/k

In [223]:
output_train=train['price']
features_query=features_test[2]
print predict_output_of_query(4, features_train, output_train, features_query)

+------------------+-------+
|     distance     | index |
+------------------+-------+
| 0.00286049526751 |  382  |
| 0.00322584038804 |  1149 |
| 0.00350215650158 |  4087 |
| 0.00359315377903 |  3142 |
| 0.00391857515673 |  2751 |
| 0.00401931061474 |  4556 |
| 0.00415816948089 |  3372 |
|  0.004185659353  |  4584 |
| 0.00426042031815 |  1666 |
| 0.00432702581392 |  2283 |
+------------------+-------+
[5527 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
[382L, 1149L, 4087L, 3142L]
413987


*** QUIZ QUESTION ***

Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

Compare this predicted value using 4-nearest neighbors to the predicted value using 1-nearest neighbor computed earlier.

## Make multiple predictions

Write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function should return a set of predicted values, one for each house in the query set.

**Hint**: To get the number of houses in the query set, use the `.shape` field of the query features matrix. See [the documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [ ]:
def predict_output(k, feature_train, output_train, features_query):
    valuesl=[]
    for i in range(len(features_query)):
        kindex=k_nearest_neighbors(k, feature_train, features_query[i])
        
        output_sum=0
        for j in kindex:
            output_sum=output_sum+output_train[j]
        valuesl.append(output_sum/k)
    return valuesl


In [252]:
output_train=train['price']
features_query=features_test[0:10]
pre= predict_output(10, features_train, output_train, features_query)
result=test['price'][0:10]
print pre
print result
rss=0
for i in range(len(pre)):
    print pre[i]
    print result[i]
    t= pre[i]-result[i]
    rss=rss+(t**2)
print rss


[881300L, 431860L, 460595L, 430200L, 766750L, 667420L, 350032L, 512800L, 484000L, 457235L]
[650000L, 485000L, 438000L, 535000L, 785000L, 975000L, 287000L, 355000L, 305000L, 518500L]
881300
650000
431860
485000
460595
438000
430200
535000
766750
785000
667420
975000
350032
287000
512800
355000
484000
305000
457235
518500
227423915774


*** QUIZ QUESTION ***

Make predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`. 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

(Depending on your computing environment, this computation may take 10-15 minutes.)

To visualize the performance as a function of `k`, plot the RSS on the VALIDATION set for each considered `k` value:

In [253]:
output_train=train['price']
vlen=len(features_valid)
features_query=features_valid[0:vlen]
rss_all=[]
for k in range(1,16):
    prediction=predict_output(k, features_train, output_train, features_query)
    rss=0
    vali=validation['price'][0:vlen]
    for i in range(vlen):
        diff=prediction[i]-vali[i]
        rss=rss+(diff**2)
    print k,rss
    rss_all.append(rss)
print rss_all

1 105468652751561
2 83445073036317
3 72692113667429
4 71934811409356
5 69777410044953
6 68899557352960
7 68338946513456
8 67372005134087
9 68372752298062
10 69332599009025
11 69523884081979
12 69051477862759
13 70011272351177
14 70911557088271
15 71110167233656
[105468652751561L, 83445073036317L, 72692113667429L, 71934811409356L, 69777410044953L, 68899557352960L, 68338946513456L, 67372005134087L, 68372752298062L, 69332599009025L, 69523884081979L, 69051477862759L, 70011272351177L, 70911557088271L, 71110167233656L]


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

kvals = range(1, 16)
plt.plot(kvals, rss_all,'bo-')

***QUIZ QUESTION ***

What is the RSS on the TEST data using the value of k found above?  To be clear, sum over all houses in the TEST set.

In [255]:
output_train=train['price']
tlen=len(features_test)
features_query=features_test[0:tlen]
pre= predict_output(8, features_train, output_train, features_query)
result=test['price'][0:tlen]
print pre
print result
rss=0
for i in range(tlen):
    print pre[i]
    print result[i]
    t= pre[i]-result[i]
    rss=rss+(t**2)
print rss

[864750L, 462700L, 394493L, 470375L, 771062L, 662118L, 343290L, 508750L, 463000L, 482856L, 462700L, 405337L, 432187L, 284875L, 758937L, 528562L, 616197L, 275687L, 562062L, 485118L, 464468L, 1009631L, 830837L, 770475L, 791250L, 414218L, 304493L, 358993L, 371056L, 405375L, 626112L, 394000L, 448812L, 1227125L, 251112L, 409431L, 434800L, 848750L, 1107062L, 413125L, 368625L, 534687L, 277756L, 2577500L, 877812L, 605500L, 556781L, 402873L, 482812L, 450062L, 463381L, 375993L, 499875L, 491437L, 388562L, 383562L, 505687L, 390665L, 453187L, 276875L, 414481L, 496668L, 383368L, 539642L, 352875L, 356000L, 414375L, 311112L, 299618L, 566615L, 428729L, 852718L, 268206L, 475181L, 226062L, 496135L, 393949L, 462056L, 388875L, 282493L, 295534L, 490931L, 433812L, 521500L, 445250L, 457125L, 581125L, 407568L, 649084L, 280812L, 1158743L, 540993L, 747521L, 1745000L, 460125L, 525324L, 202368L, 472812L, 824375L, 488368L, 559778L, 401987L, 391168L, 2895912L, 370181L, 595056L, 478000L, 678062L, 519993L, 821631L, 38

In [260]:
print '%.2E' %rss

1.33E+14
